In [ ]:
from snowflake.snowpark import Session
from snowflake.snowpark.context import get_active_session

session = get_active_session()

# Lire la table de paramétrage
param_df = session.table("TABLE_NAMES_MANPRM").select("TABLE_NAME", "FILE_FORMAT")

for row in param_df.collect():
    table_name = row["TABLE_NAME"]
    pattern_file_suffix = row["FILE_FORMAT"]

    # Construction des chemins et noms
    bronze_table = f"bronze_layer.{table_name}_BRZ"
    stage_name = "@raw_layer.landing_internal_stage"
    stage_path_suffix = bronze_table.replace('_BRZ', '/').replace('bronze_layer.', '/')
    internal_stage_asb_path = stage_name + stage_path_suffix

    external_stage_root_path = "@RAW_LAYER.EXTERNAL_AZUR_STAGE/Files"
    external_stage_abs_path = external_stage_root_path + stage_path_suffix


    pattern_file_name = f".*\\.{pattern_file_suffix.lower()}"

    try:
        copy_query = f"""
        COPY FILES
        INTO {internal_stage_asb_path}
        FROM {external_stage_abs_path}
        PATTERN = '{pattern_file_name}';
        """
        session.sql(copy_query).collect()
        print(f"Success loading files for table {table_name}")
    except Exception as e:
        print(f"Error when loading files for table {table_name}: {str(e)}")